In [1]:
import torch
from torch import nn
import numpy as np
import os,sys

In [2]:
import io

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
sys.meta_path.append(NotebookFinder())

In [4]:
import pre_process

importing Jupyter notebook from pre_process.ipynb

 [[1561, 1, 1089, 1, 102, 33, 74, 14, 7, 229, 3510, 218, 354, 4], [32, 39, 16, 10, 4904, 43, 10, 68, 44, 60, 14, 81, 2579, 0], [29, 21, 3, 225, 4, 41, 35, 93, 149, 6, 1314, 0], [71, 3, 68, 38, 40, 4, 2, 64, 0, 41, 35, 51, 106, 149, 1473, 8, 2582, 1735, 0, 1409, 177, 61, 4], [2, 325, 3, 19, 6800, 57, 654, 30, 21, 4, 2, 64, 207, 24, 1492, 36, 224, 0], [2, 746, 7, 547, 144, 6, 5, 1215, 230, 30, 23, 348, 4963, 8, 273, 56, 12, 81, 358, 0], [97, 7, 60, 228, 0, 2, 318, 476, 8, 2153, 364, 47, 46, 6, 348, 'pingpong.Perhaps', 30, 23, 106, 7, 'foursome', 28, 99, 0], [609, 133, 6, 20, 17, 140, 88, 19, 1180, 1, 30, 121, 250, 99, 6, 47, 1383, 28, 'us.That', 10, 771, 645, 8, 449, 1, 78, 0], ['Good.Let', 161, 34, 47, 92, 0], [219, 55, 0], []]

 [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1,

In [5]:
my_data = pre_process.transformed_train_dataset[0]

In [6]:
my_data['dialogue'][0][0]

[1561, 1, 1089, 1, 102, 33, 74, 14, 7, 229, 3510, 218, 354, 4]

In [7]:
my_sent = my_data['dialogue'][0]

In [8]:
my_data['dialogue'][1][0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [9]:
my_trans = my_data['dialogue'][1]

In [10]:
my_data['label'][0]

'0 0 0 0 0 0 4 4 4 4'

In [11]:
my_data['label'][1]

'3 4 2 2 2 3 4 1 3 4'

In [12]:
my_emo = my_data['label'][0]

In [13]:
my_int = my_data['label'][1]

In [14]:
len(my_emo)

19

In [15]:
my_labeling = []
i = 0
while(i < len(my_emo)):
    if(i%2==0):
        my_labeling.append(int(my_emo[i])*4 + int(my_int[i])-1)
    i = i + 1

In [16]:
my_labeling

[2, 3, 1, 1, 1, 2, 19, 16, 18, 19]

In [17]:
tor = torch.tensor(my_labeling).cuda()

In [18]:
tor

tensor([  2,   3,   1,   1,   1,   2,  19,  16,  18,  19], device='cuda:0')

In [19]:
import gensim
from gensim.models import word2vec

In [21]:
wv_model_en = word2vec.Word2Vec(size=100, window=5, min_count=5, workers=4)
wv_model_en = word2vec.Word2Vec.load('/home/jongsu/Desktop/intention_extraction/wv_parameter/dialogue_wv')

In [22]:
class chr_rnn(nn.Module):
    def __init__(self, hidden_size, bidirectional):
        super(chr_rnn, self).__init__()
        self.hidden_size = hidden_size
        if(bidirectional == True):
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.chr_rnn = nn.GRU(100, 100, bidirectional = bidirectional)
        
        
    def forward(self,char,h0):
        _,h0 = self.chr_rnn(char,h0)
        return h0
        
    def initHidden(self):
        return torch.zeros(self.bidirectional , 1, self.hidden_size, device=device)
    
    
    
    
class last_net(nn.Module):
    def __init__(self):
        super(last_net, self).__init__()
        self.lastnet = nn.Linear(200,100)
        
    def forward(self,last_hidden_state):
        last_w = self.lastnet(last_hidden_state)
        return last_w
  
    
chr_rnn_1 = chr_rnn(100, True).cuda()
last_net_1 = last_net().cuda()


In [23]:
if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_rnn_526.pth'):
    print("loading chr")
    chr_rnn_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_rnn_526.pth'))

if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_linear_526.pth'):
    print("loading last")
    last_net_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_linear_526.pth'))

In [24]:
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


In [25]:
class last_sent_net(nn.Module):
    def __init__(self):
        super(last_sent_net, self).__init__()
        self.lastnet = nn.Linear(200,100)
        
    def forward(self,last_hidden_state):
        last_w = self.lastnet(last_hidden_state)
        return last_w


In [26]:
sent = torch.tensor([[[-1.0680e-01,  4.1764e-01,  1.1729e-01,  4.8124e-02, -1.4826e-01,
           3.0498e-01,  6.3143e-02, -4.2238e-02,  2.1229e-01, -2.5284e-01,
           4.5615e-02,  2.0350e-01, -2.4534e-02, -1.2688e-01,  3.6204e-01,
          -5.2034e-02, -4.0008e-02,  1.4223e-01,  1.1551e-01, -1.5993e-01,
          -1.2564e-02, -3.6989e-02,  3.1348e-02, -1.7337e-01, -9.7827e-02,
           1.1333e-01, -2.7334e-01,  7.7455e-02, -4.5201e-02, -2.0831e-01,
          -8.4231e-02, -1.0935e-01,  7.8916e-02,  4.1244e-03,  4.2286e-01,
          -8.2768e-02,  7.7298e-02,  1.3755e-01,  8.0805e-02, -2.3777e-04,
          -6.5618e-02, -2.8547e-02, -6.1888e-02, -1.6948e-01,  1.4112e-01,
           7.1812e-02,  1.8426e-01,  8.9911e-02, -2.9633e-01, -1.5726e-02,
           5.4692e-02,  9.2683e-02,  1.5828e-01,  4.6246e-02, -1.3612e-02,
           1.3239e-01, -3.6245e-02,  1.0075e-01,  5.8007e-02,  8.8367e-02,
           7.5780e-02,  1.0242e-01, -4.9446e-02, -1.1318e-02,  1.2447e-01,
           2.0702e-02, -7.7934e-02, -1.2492e-01,  6.5921e-04, -1.5672e-01,
          -2.3884e-02, -1.6591e-01,  5.2336e-02,  1.2017e-01,  1.4578e-01,
           8.7394e-02,  1.3808e-01, -8.9080e-02,  1.3993e-01,  1.3342e-01,
          -2.6416e-01,  1.0911e-02,  1.8024e-01, -1.8492e-01, -3.3635e-02,
           2.0941e-01,  1.5569e-01,  1.3476e-02, -2.2425e-01,  6.4642e-02,
          -6.2723e-02,  4.1480e-01, -1.2245e-01, -1.2171e-01, -8.9054e-02,
          -2.1716e-01, -4.2933e-02, -1.1348e-01, -1.2962e-01,  3.6944e-02]],

        [[-1.2018e-01,  1.8581e-01,  1.0395e-01, -5.2564e-02, -1.3379e-01,
           2.1374e-01,  1.3466e-01, -1.0585e-01,  1.4960e-01, -2.7677e-01,
           1.0774e-02,  1.3158e-01,  4.1090e-02,  4.8667e-02,  2.5489e-01,
           3.1143e-02,  4.0489e-02,  5.1822e-02,  2.6525e-01, -2.0009e-01,
           6.1175e-02, -3.4685e-02, -6.9980e-03, -1.2247e-01, -1.8888e-02,
           1.1724e-01, -2.6646e-01,  1.4652e-01, -1.6050e-01, -1.5565e-01,
          -1.0895e-01, -5.5406e-02,  1.4598e-01, -4.9225e-02,  2.7255e-01,
          -1.7466e-01,  3.3056e-02,  1.8309e-01, -6.0993e-02, -2.5510e-02,
          -8.0520e-03,  9.8644e-02, -1.3170e-02, -1.6070e-01,  2.7834e-02,
           6.1881e-02,  7.5724e-02,  4.2528e-02, -2.3503e-01,  3.9188e-02,
           8.6388e-02,  1.5592e-01,  1.3454e-01,  7.5253e-02,  3.3208e-02,
           1.8307e-01,  5.6556e-02,  1.5848e-02,  1.2437e-01,  1.5498e-02,
           7.8508e-03,  2.4991e-02, -1.8756e-02,  3.7022e-02,  5.7623e-02,
          -2.3879e-02, -4.4766e-02, -4.4164e-02, -3.6912e-02, -7.3080e-02,
          -5.2377e-02, -1.4912e-01, -4.1641e-02,  1.1988e-01,  2.5016e-01,
          -6.4747e-02,  7.3905e-02, -1.0514e-01,  1.1824e-01,  6.7400e-02,
          -1.7377e-01,  2.7934e-02,  1.2910e-01, -6.4466e-03, -4.5180e-02,
           1.1318e-01,  2.1908e-01, -3.2531e-02, -7.7870e-02,  8.3792e-03,
          -1.3692e-01,  3.7378e-01, -4.4081e-02, -1.3466e-01,  7.1220e-02,
          -1.6950e-01, -5.0803e-03,  4.0684e-03, -1.1930e-01,  1.6220e-01]],

        [[-9.8497e-02,  2.6200e-01,  1.4556e-01,  1.4029e-02, -1.4945e-01,
           2.3177e-01,  1.4858e-01, -6.6958e-02,  1.3959e-01, -2.5700e-01,
           4.3991e-02,  1.4906e-01, -4.9267e-03, -1.3645e-02,  2.9141e-01,
           1.7603e-02,  1.9018e-02,  7.3066e-02,  2.8715e-01, -1.8885e-01,
           6.5032e-02, -8.5621e-02,  7.4093e-03, -1.1440e-01, -5.2371e-02,
           1.3332e-01, -3.2457e-01,  1.1822e-01, -2.3088e-01, -1.5293e-01,
          -1.0059e-01, -5.8964e-02,  1.6911e-01, -3.1601e-04,  2.9812e-01,
          -1.9797e-01,  5.2667e-03,  1.7787e-01, -8.0930e-02, -3.7694e-02,
          -5.4866e-02,  4.1305e-02, -1.8542e-02, -1.3595e-01,  6.4230e-02,
           1.2351e-01,  8.3782e-02,  3.9007e-02, -2.1733e-01, -2.5266e-02,
           1.0495e-01,  1.5829e-01,  1.5822e-01,  3.3002e-02,  8.1013e-02,
           1.8979e-01,  4.7653e-02,  7.6019e-02,  8.1035e-02,  5.5273e-02,
           7.0763e-04,  2.1019e-02, -5.5004e-02,  1.0529e-02,  5.8949e-02,
          -2.9307e-03, -8.7908e-02, -3.5904e-02, -4.3027e-02, -1.1385e-01,
          -3.5690e-02, -1.4619e-01, -5.4781e-02,  1.4359e-01,  2.3254e-01,
          -4.4227e-02,  4.0230e-02, -7.2305e-02,  1.4932e-01,  6.3454e-02,
          -1.8485e-01,  8.3879e-02,  1.0001e-01, -1.4474e-02, -7.4540e-02,
           1.3394e-01,  2.2892e-01, -3.0534e-02, -9.0421e-02,  3.3334e-03,
          -1.5153e-01,  3.8708e-01, -4.1677e-02, -1.1780e-01,  6.1460e-02,
          -1.3994e-01,  2.0478e-02, -7.3367e-02, -1.3555e-01,  1.7400e-01]],

        [[-1.1508e-01,  4.2273e-01,  9.5441e-02,  3.5970e-02, -1.4525e-01,
           2.8670e-01,  6.2170e-02, -5.5618e-02,  2.1610e-01, -2.6093e-01,
           5.3586e-02,  2.1109e-01, -4.2897e-02, -1.1414e-01,  3.6680e-01,
          -4.7859e-02, -5.1719e-02,  1.4759e-01,  9.4465e-02, -1.6045e-01,
          -1.0217e-02, -4.7173e-02,  3.8518e-02, -1.6793e-01, -9.3243e-02,
           1.3887e-01, -2.6538e-01,  8.4276e-02, -2.0503e-02, -2.1999e-01,
          -7.0604e-02, -9.5037e-02,  8.6217e-02, -2.2933e-03,  4.2951e-01,
          -6.4124e-02,  5.9110e-02,  1.3890e-01,  7.0494e-02,  1.2604e-02,
          -4.8436e-02, -1.6203e-02, -5.5258e-02, -1.5859e-01,  1.3959e-01,
           7.3348e-02,  1.9342e-01,  1.1537e-01, -3.0680e-01, -3.6700e-03,
           3.1957e-02,  7.4969e-02,  1.5231e-01,  6.9952e-02, -3.1404e-02,
           1.3079e-01, -4.5337e-02,  9.0991e-02,  7.0794e-02,  8.5108e-02,
           7.3199e-02,  8.7299e-02, -3.5193e-02, -8.4781e-03,  1.1855e-01,
           2.8050e-02, -7.5971e-02, -1.5388e-01, -2.9153e-03, -1.5551e-01,
          -2.2781e-02, -1.7722e-01,  6.9022e-02,  1.0048e-01,  1.4665e-01,
           9.6696e-02,  1.2916e-01, -6.2130e-02,  1.5789e-01,  1.2334e-01,
          -2.7109e-01, -1.0784e-02,  2.0572e-01, -1.9972e-01, -2.0150e-02,
           2.1259e-01,  1.5013e-01,  3.5308e-02, -2.4015e-01,  6.0764e-02,
          -5.7596e-02,  4.2861e-01, -1.5821e-01, -1.1614e-01, -1.0283e-01,
          -2.1300e-01, -6.0537e-02, -1.0794e-01, -1.3117e-01, -5.2044e-03]],

        [[-1.5853e-01,  2.2225e-01,  1.0330e-01, -5.6842e-02, -1.7970e-01,
           2.4980e-01,  1.8117e-01, -5.6338e-02,  2.1726e-01, -3.1263e-01,
           6.7152e-03,  1.6242e-01,  2.7603e-02, -3.0384e-04,  3.2019e-01,
           3.7633e-02,  4.4230e-02,  6.6146e-02,  2.8802e-01, -2.1292e-01,
           4.6246e-02, -4.1872e-02, -3.5669e-02, -1.1110e-01, -1.0262e-02,
           1.5739e-01, -3.0178e-01,  1.5285e-01, -1.9618e-01, -1.7792e-01,
          -1.0969e-01, -3.9697e-02,  1.4227e-01, -2.0163e-02,  3.5303e-01,
          -1.9160e-01,  1.4430e-02,  2.2094e-01, -5.4493e-02, -4.1510e-02,
          -4.7131e-02,  8.4856e-02, -3.1494e-02, -1.8457e-01,  5.8469e-02,
           1.1025e-01,  9.8716e-02,  3.3137e-02, -2.9168e-01,  1.6809e-02,
           7.3728e-02,  1.7640e-01,  1.4856e-01,  5.2897e-02,  5.4147e-02,
           2.2815e-01,  3.4194e-02,  5.6142e-02,  1.4287e-01,  5.4549e-02,
           1.9337e-02,  1.7131e-02, -3.7440e-02,  4.6957e-02,  5.3956e-02,
          -2.6943e-02, -5.8819e-02, -5.8320e-02, -4.0830e-02, -8.7833e-02,
          -3.6543e-02, -1.8613e-01, -3.0790e-02,  1.3175e-01,  2.7497e-01,
          -5.4593e-02,  4.5633e-02, -9.6054e-02,  1.5312e-01,  7.5317e-02,
          -2.3315e-01,  4.9053e-02,  1.1266e-01, -2.1935e-02, -9.3054e-02,
           1.2296e-01,  2.1635e-01, -4.3962e-02, -7.2645e-02,  2.0075e-02,
          -1.7470e-01,  4.0869e-01, -5.7928e-02, -1.4400e-01,  8.8183e-02,
          -1.8635e-01,  5.8732e-03, -2.7454e-02, -1.0333e-01,  1.7986e-01]],

        [[-1.5408e-01,  2.4032e-01,  1.0362e-01, -6.4615e-02, -1.5186e-01,
           2.4547e-01,  1.3608e-01, -9.8447e-02,  2.0839e-01, -3.3073e-01,
           3.7946e-03,  2.1075e-01,  5.4739e-02,  1.4090e-02,  3.1261e-01,
           2.4893e-02,  3.0926e-02,  7.1692e-02,  2.7876e-01, -1.8573e-01,
           2.9103e-02, -5.6659e-02, -1.1711e-02, -1.3038e-01, -4.3316e-02,
           1.2663e-01, -3.0980e-01,  1.4935e-01, -1.5367e-01, -1.8468e-01,
          -1.1012e-01, -5.8075e-02,  1.3262e-01, -5.6496e-02,  3.7462e-01,
          -1.8124e-01,  6.4991e-02,  2.1003e-01, -4.6860e-02, -5.3153e-02,
          -2.4518e-02,  6.1324e-02, -1.9072e-02, -1.6827e-01,  3.3081e-02,
           7.0777e-02,  1.1658e-01,  6.0550e-02, -2.7161e-01, -2.9951e-03,
           1.0193e-01,  1.5739e-01,  1.4767e-01,  7.3332e-02,  1.1104e-02,
           2.2503e-01,  3.9120e-02,  6.2831e-02,  1.6325e-01,  4.9862e-02,
          -5.1409e-07,  2.9847e-02, -4.9078e-02,  3.5096e-02,  8.2957e-02,
          -4.9048e-02, -4.7899e-02, -9.0792e-02, -5.2271e-02, -9.4259e-02,
          -4.2930e-02, -1.7699e-01, -1.2347e-03,  1.1002e-01,  2.5640e-01,
          -5.7281e-02,  7.0602e-02, -9.8228e-02,  1.5336e-01,  6.4138e-02,
          -2.2282e-01,  3.4397e-02,  1.1678e-01, -4.9629e-02, -9.0187e-02,
           1.5823e-01,  2.2401e-01, -5.8777e-02, -7.6672e-02,  5.9465e-02,
          -1.3551e-01,  4.2273e-01, -8.7628e-02, -1.2685e-01,  6.5579e-02,
          -1.7320e-01, -1.4559e-02, -5.7226e-03, -1.1088e-01,  1.5411e-01]],

        [[-1.4391e-01,  2.7251e-01,  1.1570e-01, -2.7448e-02, -1.7658e-01,
           2.8328e-01,  1.6038e-01, -3.3116e-02,  2.3003e-01, -3.1616e-01,
           2.2096e-02,  1.6179e-01,  1.3403e-02, -4.1768e-02,  3.3790e-01,
           3.3000e-02,  1.7191e-02,  6.5023e-02,  2.6911e-01, -2.0858e-01,
           4.2974e-02, -6.5912e-02, -2.1549e-02, -1.1182e-01, -4.4112e-02,
           1.4746e-01, -3.1225e-01,  1.5728e-01, -1.7570e-01, -1.8402e-01,
          -1.1712e-01, -6.8455e-02,  1.4909e-01, -6.5313e-03,  3.8800e-01,
          -1.8849e-01,  4.2664e-02,  2.2824e-01, -2.0789e-02, -4.6852e-02,
          -7.3238e-02,  4.1998e-02, -2.4564e-02, -1.7720e-01,  6.4457e-02,
           1.0295e-01,  1.2569e-01,  2.9976e-02, -2.6674e-01, -2.1509e-02,
           1.0175e-01,  1.7626e-01,  1.6870e-01,  3.2110e-02,  5.7414e-02,
           2.2310e-01,  2.1346e-02,  7.9639e-02,  1.0638e-01,  9.1218e-02,
           1.6387e-02,  2.8936e-02, -5.6522e-02,  3.8876e-02,  9.3165e-02,
          -1.8086e-02, -6.2638e-02, -7.0345e-02, -4.1194e-02, -9.9076e-02,
          -3.0279e-02, -1.6334e-01, -1.7999e-02,  1.5583e-01,  2.5865e-01,
          -2.3955e-02,  5.7950e-02, -8.6155e-02,  1.6193e-01,  7.1474e-02,
          -2.3868e-01,  6.5018e-02,  9.9212e-02, -5.4617e-02, -1.1543e-01,
           1.4249e-01,  2.1768e-01, -3.7782e-02, -8.8439e-02,  2.5411e-02,
          -1.4875e-01,  4.0775e-01, -5.8322e-02, -1.3008e-01,  5.6311e-02,
          -1.6477e-01, -9.7234e-03, -6.1942e-02, -1.1475e-01,  1.8431e-01]],

        [[-1.6388e-01,  1.8043e-01,  9.6969e-02, -4.1949e-02, -1.7415e-01,
           2.7098e-01,  1.9539e-01, -6.5586e-02,  2.1534e-01, -2.9011e-01,
          -3.9719e-02,  1.1360e-01,  8.3059e-02,  9.5495e-03,  3.1754e-01,
           8.8558e-02,  8.4038e-02,  3.1366e-02,  2.9906e-01, -2.5275e-01,
           1.1987e-01,  1.5585e-02, -1.0218e-01, -1.6007e-01,  1.8973e-02,
           1.8002e-01, -3.0540e-01,  1.5588e-01, -1.5980e-01, -1.6095e-01,
          -1.3758e-01, -3.4839e-02,  1.9381e-01, -3.2990e-02,  2.7224e-01,
          -1.1413e-01, -1.4072e-03,  2.2545e-01,  3.8254e-03,  2.3968e-02,
          -1.4079e-02,  1.0683e-01, -2.4544e-02, -1.6082e-01,  4.3283e-02,
           6.6874e-02,  5.6874e-02,  1.8293e-02, -3.1524e-01,  1.1451e-01,
           6.7172e-02,  1.8395e-01,  1.7244e-01,  6.9279e-02,  9.0909e-02,
           1.9762e-01,  4.5253e-02, -8.0515e-03,  1.6161e-01,  4.8623e-02,
           1.7687e-02,  1.8199e-02, -4.2223e-03,  1.1595e-01,  3.3348e-02,
          -2.4794e-02, -8.3397e-02, -8.3020e-02, -3.7966e-03, -5.3187e-02,
          -2.4382e-02, -2.3012e-01, -2.8697e-02,  1.0883e-01,  3.3588e-01,
          -1.4239e-01,  7.6649e-02, -1.3704e-01,  9.5923e-02,  9.5764e-02,
          -2.2307e-01,  2.2441e-02,  7.0683e-02,  2.6640e-02, -7.9979e-02,
           4.2144e-02,  2.6405e-01, -5.3706e-02, -1.3995e-01,  3.2031e-03,
          -2.3390e-01,  4.2133e-01, -7.6724e-02, -1.4652e-01,  9.2589e-02,
          -2.1704e-01,  3.8315e-02,  6.8709e-02, -1.8788e-01,  2.0672e-01]],

        [[-1.2824e-01,  2.0264e-01,  7.2541e-02, -6.2766e-02, -1.8653e-01,
           2.2835e-01,  2.0552e-01, -3.4741e-02,  2.4176e-01, -2.9545e-01,
           3.0598e-03,  1.5691e-01,  2.3333e-03,  1.2443e-03,  3.2266e-01,
           4.2655e-02,  3.6637e-02,  6.2943e-02,  2.7011e-01, -2.1093e-01,
           5.5329e-02,  1.6563e-03, -7.5484e-02, -8.4686e-02,  1.9288e-03,
           1.5337e-01, -2.6956e-01,  1.4858e-01, -1.5631e-01, -1.6954e-01,
          -9.6037e-02, -5.5620e-02,  9.0198e-02,  2.5982e-02,  3.6339e-01,
          -1.5943e-01, -9.8635e-03,  1.9461e-01,  8.2307e-03, -1.7162e-02,
          -7.4801e-02,  5.8591e-02, -3.8403e-02, -2.1066e-01,  5.5701e-02,
           1.5606e-01,  7.9197e-02,  2.5202e-02, -3.0487e-01,  6.0959e-02,
           7.4710e-02,  1.4459e-01,  1.4230e-01, -6.9683e-03,  7.7575e-02,
           2.6138e-01,  3.4847e-02,  2.1499e-02,  1.4424e-01,  5.7442e-02,
           4.7805e-02,  3.9731e-02,  2.1508e-02,  1.1571e-01,  1.1776e-02,
           1.2663e-02, -7.9326e-02, -5.3951e-02, -4.8212e-02, -1.8787e-02,
           2.0211e-02, -1.8490e-01, -4.8212e-02,  8.4903e-02,  2.4211e-01,
          -4.5050e-02,  2.4055e-02, -8.9620e-02,  1.6395e-01,  8.3441e-02,
          -2.7449e-01,  2.6320e-02,  9.2947e-02, -3.1645e-02, -9.2916e-02,
           1.2633e-01,  1.4438e-01, -7.4435e-02, -2.2577e-02, -8.3080e-04,
          -2.0007e-01,  3.8993e-01, -5.0673e-02, -1.4184e-01,  8.3925e-02,
          -1.7746e-01,  2.4870e-03, -3.1859e-02, -1.4627e-01,  1.5149e-01]],

        [[-1.2182e-01,  1.6747e-01,  5.1555e-02, -3.2861e-02, -1.2799e-01,
           2.1062e-01,  1.9250e-01, -4.2417e-02,  2.1313e-01, -2.2892e-01,
          -6.6504e-02,  1.1224e-01,  3.1424e-02,  1.6433e-02,  2.7538e-01,
           1.8132e-02,  5.4086e-03, -3.8532e-02,  2.3523e-01, -2.1624e-01,
           6.0496e-02,  3.8744e-02, -9.5755e-02, -1.2956e-01,  2.8911e-02,
           1.4867e-01, -2.5475e-01,  1.7615e-01, -1.0655e-01, -1.2834e-01,
          -7.4087e-02, -3.5754e-02,  1.1259e-01,  4.1909e-03,  2.8379e-01,
          -1.1619e-01,  2.8672e-04,  1.7785e-01,  3.8190e-02,  2.1272e-02,
          -3.1597e-02,  1.3703e-01, -1.6237e-02, -1.8118e-01,  2.7387e-02,
           7.6947e-02,  4.3289e-02,  3.4056e-02, -2.4057e-01,  9.8319e-02,
           2.2994e-02,  9.3788e-02,  1.5602e-01,  1.7743e-02,  8.9378e-02,
           1.8941e-01,  3.8036e-02,  1.8984e-02,  1.2595e-01,  9.2422e-03,
           4.8488e-02,  5.6824e-02,  4.8577e-02,  1.4668e-01,  2.8725e-02,
          -3.1731e-02, -8.2385e-02, -7.8605e-02, -7.9422e-02, -4.2988e-02,
          -1.0066e-03, -1.2511e-01,  8.2135e-03,  1.0117e-01,  2.5657e-01,
          -3.8665e-02,  5.6144e-02, -7.7556e-02,  1.1775e-01,  1.0016e-01,
          -2.4695e-01,  2.8276e-02,  7.7958e-02, -7.7261e-03, -6.0247e-02,
           5.0185e-02,  1.3726e-01, -4.3570e-02, -1.1779e-01, -3.4677e-02,
          -1.6387e-01,  3.5505e-01, -6.2150e-02, -1.5702e-01,  3.2642e-02,
          -2.1101e-01,  1.6306e-02,  1.1516e-02, -1.8863e-01,  1.3429e-01]]], device='cuda:0')

In [27]:
criterian = nn.CrossEntropyLoss()


class Gru(nn.Module):
    
    def __init__(self, hidden_dim):
        super(Gru, self).__init__()
        self.gru = nn.GRU(100,100,2,bidirectional = True)
        self.inh = torch.zeros(4, 1, 100).cuda()
        self.nn = nn.Linear(200,28)

        self.softmax = nn.Softmax(dim = 1)
        
    def init_hidden(self):
        return torch.zeros(4, 1, 100).cuda()
            
    
    def forward(self, sentence):  
        
        output,hidden = self.gru(sentence,self.inh)
        output = self.nn(output.view(len(sentence),200))
        output2 = self.softmax(output)
        #print("bfs22222222222222222222222222222 \n",output2)
        return output2

In [63]:
test1 = Gru(100).cuda()

In [78]:
from torch import optim
optimizer1 = optim.SGD(test1.parameters(), lr=0.0001)#, weight_decay=1e-4)

In [79]:
i = 0
while i < 1000:
    optimizer1.zero_grad()
    i = i + 1
    newtag = test1(sent)
    loss = criterian(newtag, tor)
    loss.backward()
    optimizer1.step()
    #print(loss)
    
#tensor([  2,   3,   1,   1,   1,   2,  19,  16,  18,  19], device='cuda:0')

In [80]:
print (newtag)

tensor([[ 0.0054,  0.8485,  0.0180,  0.0069,  0.0046,  0.0051,  0.0043,
          0.0042,  0.0047,  0.0053,  0.0045,  0.0042,  0.0046,  0.0044,
          0.0033,  0.0047,  0.0083,  0.0041,  0.0048,  0.0095,  0.0048,
          0.0049,  0.0055,  0.0056,  0.0048,  0.0053,  0.0047,  0.0049],
        [ 0.0026,  0.9245,  0.0103,  0.0035,  0.0022,  0.0025,  0.0020,
          0.0020,  0.0024,  0.0025,  0.0020,  0.0020,  0.0022,  0.0021,
          0.0015,  0.0022,  0.0045,  0.0019,  0.0025,  0.0054,  0.0022,
          0.0024,  0.0026,  0.0028,  0.0022,  0.0025,  0.0022,  0.0022],
        [ 0.0015,  0.9526,  0.0071,  0.0022,  0.0013,  0.0016,  0.0011,
          0.0012,  0.0015,  0.0015,  0.0012,  0.0012,  0.0013,  0.0013,
          0.0009,  0.0013,  0.0030,  0.0011,  0.0016,  0.0037,  0.0013,
          0.0015,  0.0016,  0.0017,  0.0013,  0.0015,  0.0013,  0.0013],
        [ 0.0012,  0.9634,  0.0057,  0.0017,  0.0010,  0.0012,  0.0008,
          0.0009,  0.0012,  0.0011,  0.0009,  0.0009,  0.0010